In [1]:
import requests
import random
import pandas as pd
from bs4 import BeautifulSoup
from headers import headers_list
from loading_bar import log_progress

In [2]:
url = 'https://graduan.com/accenture'
url

'https://graduan.com/accenture'

In [11]:
def get_company_info(url):
    company = {}
    page = requests.get(url, headers=random.choice(headers_list))
    soup = BeautifulSoup(page.content, 'html.parser')
    info = soup.find('div', class_='small-section')
    for div in info.findAll('div'):
        head = div.find('h4')
        if head is not None:
            head = head.text
            # Contact number
            if head == 'Contact Number':
                a = div.find('a')
                if a is not None:
                    body = a['href'].strip('\n')
                    company['Telephone'] = body
                try:
                    company['Contact Person'] = div.text.split('-')[-1]
                except:
                    pass
            else:
                a = div.find('a')
                if a is not None:
                    body = a['href'].strip('\n')
                else:
                    body = div.text.replace(head, '').strip('\n')
                company[head] = body
    return company

In [12]:
get_company_info(url)

{'Telephone': 'tel:03 2088 4000',
 'Contact Person': ' Ju Min Lim\n\n',
 'Fax No.': '03 20887000',
 'Email': '/cdn-cgi/l/email-protection#e4a3968b93b38d908cb197a4858787818a90919681ca878b89',
 'Address': 'Level 30 Menara Exchange 106, Lingkaran TRX Tun Razak Exchange, 55188 Kuala Lumpur.'}

In [13]:
def get_companies_from_page(url):
    companies = []
    page = requests.get(url, headers=random.choice(headers_list))
    soup = BeautifulSoup(page.content, 'html.parser')
    info = soup.find('div', class_='card-info-content')
    for card in info.findAll('div', class_='card-box-wrap'):
        a = card.find('a')
        if a is not None:
            company = get_company_info(a['href'])
            company['Company'] = a.text.strip('\n')
            companies.append(company)
    return companies

In [14]:
url = 'https://graduan.com/companies'
get_companies_from_page(url)

[{'Telephone': 'tel:03 2088 4000',
  'Contact Person': ' Ju Min Lim\n\n',
  'Fax No.': '03 20887000',
  'Email': '/cdn-cgi/l/email-protection#84c3f6ebf3d3edf0ecd1f7c4e5e7e7e1eaf0f1f6e1aae7ebe9',
  'Address': 'Level 30 Menara Exchange 106, Lingkaran TRX Tun Razak Exchange, 55188 Kuala Lumpur.',
  'Company': 'ACCENTURE'},
 {'Telephone': 'tel:+6565663311',
  'Contact Person': ' Chuah Xing Mei\n\n',
  'Fax No.': '+6565677365',
  'Email': '/cdn-cgi/l/email-protection#81e2e9f4e0e9f9e8efe6ece4e8c1e0e2f4ece4e5afe2eeecaff2e6',
  'Address': '10 Toh Guan Road #09-02 Singapore 608838',
  'Company': 'Acumed Medical Group'},
 {'Telephone': 'tel:03-76613636',
  'Contact Person': '\n\n',
  'Address': 'AEGIS BPO Malaysia Sdn Bhd, \nLevel 2, Symphony House,\nPusat Dagangan Dana 1 JalanPJU 1A/46\n47301 Petaling Jaya\nSelangor, Malaysia',
  'Company': 'AEGIS BPO Malaysia Sdn Bhd'},
 {'Telephone': 'tel:03-9207 2107',
  'Contact Person': ' Pravina Karpan\n\n',
  'Fax No.': '03-9207 2006/2007',
  'Address': 

In [15]:
base_url = 'https://graduan.com/companies?page={}'
start = 1
end = 15
COMPANY_LIST = []

for i in log_progress(range(start, end+1)):
    url = base_url.format(i)
    companies = get_companies_from_page(url)
    COMPANY_LIST = COMPANY_LIST + companies

In [16]:
COMPANY_LIST

[{'Telephone': 'tel:03 2088 4000',
  'Contact Person': ' Ju Min Lim\n\n',
  'Fax No.': '03 20887000',
  'Email': '/cdn-cgi/l/email-protection#6126130e1636081509341221000202040f151413044f020e0c',
  'Address': 'Level 30 Menara Exchange 106, Lingkaran TRX Tun Razak Exchange, 55188 Kuala Lumpur.',
  'Company': 'ACCENTURE'},
 {'Telephone': 'tel:+6565663311',
  'Contact Person': ' Chuah Xing Mei\n\n',
  'Fax No.': '+6565677365',
  'Email': '/cdn-cgi/l/email-protection#05666d70646d7d6c6b6268606c456466706860612b666a682b7662',
  'Address': '10 Toh Guan Road #09-02 Singapore 608838',
  'Company': 'Acumed Medical Group'},
 {'Telephone': 'tel:03-76613636',
  'Contact Person': '\n\n',
  'Address': 'AEGIS BPO Malaysia Sdn Bhd, \nLevel 2, Symphony House,\nPusat Dagangan Dana 1 JalanPJU 1A/46\n47301 Petaling Jaya\nSelangor, Malaysia',
  'Company': 'AEGIS BPO Malaysia Sdn Bhd'},
 {'Telephone': 'tel:03-9207 2107',
  'Contact Person': ' Pravina Karpan\n\n',
  'Fax No.': '03-9207 2006/2007',
  'Address': 

In [18]:
df = pd.DataFrame.from_dict(COMPANY_LIST)
df = df[['Company', 'Contact Person', 'Address', 'Email', 'Telephone', 'Fax No.']]
df.head()

,Company,Contact Person,Address,Email,Telephone,Fax No.
0,ACCENTURE,Ju Min Lim\n\n,"Level 30 Menara Exchange 106, Lingkaran TRX Tu...",/cdn-cgi/l/email-protection#6126130e1636081509...,tel:03 2088 4000,03 20887000
1,Acumed Medical Group,Chuah Xing Mei\n\n,10 Toh Guan Road #09-02 Singapore 608838,/cdn-cgi/l/email-protection#05666d70646d7d6c6b...,tel:+6565663311,+6565677365
2,AEGIS BPO Malaysia Sdn Bhd,\n\n,"AEGIS BPO Malaysia Sdn Bhd, \nLevel 2, Symphon...",NaN,tel:03-76613636,NaN
3,AEON CO. (M) BHD.,Pravina Karpan\n\n,"3rd Floor, AEON Taman Maluri Shopping Centre\n...",NaN,tel:03-9207 2107,03-9207 2006/2007
4,AEON CREDIT SERVICE (M) BERHAD,Munirah\n\n,"Level 18, UOA Corporate Tower\nAvenue 10, The ...",/cdn-cgi/l/email-protection#10637964797e65627d...,tel:03 2772 9306,NaN


In [27]:
def clean_person(person):
    if not isinstance(person, str):
        return None
    person = person.replace('\n', '').strip()
    if len(person) <= 1:
        return None
    return person

def clean_number(num):
    if not isinstance(num, str):
        return None
    num = num.replace('tel:', '')
    return num

def decode_email(email):
    if not isinstance(email, str):
        return None
    e = email.split('#')[-1]
    de = ''
    k = int(e[:2], 16)
    for i in range(2, len(e)-1, 2):
        de += chr(int(e[i:i+2], 16)^k)
    return de

In [32]:
# Data cleaning
df_output = df.copy()
df_output['Contact Person'] = df_output['Contact Person'].apply(lambda x: clean_person(x))
df_output['Address'] = df_output['Address'].str.replace('\n', ' ', regex=True)
df_output['Email'] = df_output['Email'].apply(lambda x: decode_email(x))
df_output['Telephone'] = df_output['Telephone'].apply(lambda x: clean_number(x))
df_output.head()

,Company,Contact Person,Address,Email,Telephone,Fax No.
0,ACCENTURE,Ju Min Lim,"Level 30 Menara Exchange 106, Lingkaran TRX Tu...",GrowWithUs@accenture.com,03 2088 4000,03 20887000
1,Acumed Medical Group,Chuah Xing Mei,10 Toh Guan Road #09-02 Singapore 608838,chuahxingmei@acumed.com.sg,+6565663311,+6565677365
2,AEGIS BPO Malaysia Sdn Bhd,None,"AEGIS BPO Malaysia Sdn Bhd, Level 2, Symphony...",None,03-76613636,NaN
3,AEON CO. (M) BHD.,Pravina Karpan,"3rd Floor, AEON Taman Maluri Shopping Centre J...",None,03-9207 2107,03-9207 2006/2007
4,AEON CREDIT SERVICE (M) BERHAD,Munirah,"Level 18, UOA Corporate Tower Avenue 10, The V...",sitinurmunirah@aeoncredit.com.my,03 2772 9306,NaN


In [33]:
len(df_output)

294

In [34]:
df_output.to_csv('results/graduan.csv', index=False)